In [14]:
import pandas as pd
import os
import zipfile
import requests
from io import BytesIO
from utils.save_csv import save_csv
from utils.geoloc import azure_maps_address_search
from config import data_folder

In [15]:
file_tse_locais = os.path.join(data_folder, 'locais_votacao.csv')

url_tse_locais = 'https://cdn.tse.jus.br/estatistica/sead/odsele/detalhe_votacao_secao/detalhe_votacao_secao_2024.zip'

In [16]:
def load_csv_votacao(url = url_tse_locais, file_name=file_tse_locais):

    print('Loading TSE voting locations data...')
    response = requests.get(url)

    with zipfile.ZipFile(BytesIO(response.content)) as z:

        csv_name = [f for f in z.namelist() if 'SP' in f and f.endswith('.csv')][0]

        with z.open(csv_name) as f:

            locais_votacao = pd.read_csv(f, 
                                         sep = ';', 
                                         encoding = 'latin1')

    save_csv(locais_votacao, file_name)

    return locais_votacao

In [17]:
if os.path.exists(file_tse_locais):

    locais_votacao = pd.read_csv(file_tse_locais,
                                 sep = ';', 
                                 encoding = 'utf-8')

else:
    print('Loading saved csv.')
    locais_votacao=load_csv_votacao()

In [18]:
locais_votacao_sp = locais_votacao[locais_votacao['NM_MUNICIPIO'] == 'SÃO PAULO']

In [19]:
locais_votacao_sp['NR_ZONA'].unique()

array([260, 246, 381, 389, 280, 346, 327, 374, 257, 403, 253, 392, 372,
       249, 376, 258, 353, 371, 421,   2, 348, 390, 349, 420, 326, 404,
       251,  20, 320, 413, 417, 254, 408, 328, 373, 325, 397, 405, 255,
       375, 418, 350, 256, 250, 347, 352,   6, 422,   4, 248,   3, 252,
         1, 351, 247,   5, 259])

In [20]:
locais_votacao_sp['NR_SECAO'].unique()

array([ 497,  499,  309, ..., 1049, 1057, 1034], shape=(1057,))

In [21]:
locais_votacao_sp['DS_LOCAL_VOTACAO_ENDERECO'].unique()

array(['RUA ALENCAR ARARIPE, 261', 'RUA CAMPANTE, 357',
       'RUA VERBO DIVINO, 993 A', ..., 'RUA RÔMULO NALDI, 117',
       'RUA ANTONIO MARIANI, 425', 'AV. NAÇÕES UNIDAS, 1525'],
      shape=(2033,), dtype=object)

In [22]:
resp = azure_maps_address_search('Rua dos Bebados, 0 ')

Searching Azure Maps:  https://atlas.microsoft.com/search/address/structured/json?streetName=Rua+dos+Bebados&streetNumber=0&municipality=Sao+Paulo&countrySecondarySubdivision=Sao+Paulo&countryCode=br&api-version=1.0&language=en-US&subscription-key=**TOKEN_STRING**


In [23]:
resp.keys()

dict_keys(['summary', 'results'])

In [24]:
resp['summary']

{'query': 'rua dos bebados 0 sao paulo sao paulo',
 'queryType': 'NON_NEAR',
 'queryTime': 174,
 'numResults': 10,
 'offset': 0,
 'totalResults': 10245,
 'fuzzyLevel': 2}

In [25]:
len(resp['results'])

10

In [26]:
resp['results'][0]

{'type': 'Street',
 'id': '3wJvttS4Lp9LNplNq2e3Qw',
 'score': 0.5089584468825264,
 'matchConfidence': {'score': 0.5089584468825264},
 'address': {'streetName': 'Rua dos Patis',
  'municipalitySubdivision': 'Cachoeirinha',
  'municipality': 'São Paulo',
  'countrySubdivision': 'São Paulo',
  'countrySubdivisionName': 'São Paulo',
  'countrySubdivisionCode': 'SP',
  'countryCode': 'BR',
  'country': 'Brazil',
  'countryCodeISO3': 'BRA',
  'freeformAddress': 'Rua dos Patis, São Paulo',
  'localName': 'São Paulo'},
 'position': {'lat': -23.4714757, 'lon': -46.6611156},
 'viewport': {'topLeftPoint': {'lat': -23.46786, 'lon': -46.66331},
  'btmRightPoint': {'lat': -23.47518, 'lon': -46.65858}}}